In [ ]:
# Install required libraries
!pip install transformers datasets torch scikit-learn sentencepiece faker gradio

# Import Libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from datasets import Dataset
import pandas as pd
from faker import Faker
import random
import gradio as gr  #Gradio for UI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Check device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Initialize model and tokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# Initialize Faker
fake = Faker()
Faker.seed(42)

# Updated Dataset Generation Function
def generate_sample_dataset(num_samples=100):
    data = []
    plagiarism_types = ['verbatim', 'paraphrase', 'cross_language', 'ai_generated', 'none']

    for _ in range(num_samples):
        original = fake.paragraph(nb_sentences=5) + " " + fake.sentence()
        source_doc = {
            'text': original,
            'metadata': {
                'paper_id': fake.uuid4(),
                'discipline': random.choice(['cs', 'physics', 'biology', 'linguistics']),
                'publication_year': random.randint(2010, 2023)
            }
        }

        # Generate suspicious document
        plag_type = random.choice(plagiarism_types)
        if plag_type == 'none':
            suspicious = fake.paragraph(nb_sentences=5) + " " + fake.sentence()
            label = 0
        else:
            if plag_type == 'verbatim':
                if random.random() > 0.5:
                    suspicious = original  # Full verbatim copy
                else:
                    suspicious = original[:int(len(original)*0.8)]  # 80% verbatim copy
            elif plag_type == 'paraphrase':
                suspicious = original.replace("important", "crucial").replace("method", "approach")
            elif plag_type == 'cross_language':
                suspicious = original + " " + fake.paragraph(nb_sentences=2)
            elif plag_type == 'ai_generated':
                suspicious = original.upper() if random.random() > 0.5 else original.lower()
            label = 1

        data.append({
            'text1': original,
            'text2': suspicious,
            'label': label,
            'plagiarism_type': plag_type,
            'source_document_id': source_doc['metadata']['paper_id'],
            'suspicious_document_id': fake.uuid4()
        })

    return Dataset.from_pandas(pd.DataFrame(data))

# Generate dataset
dataset = generate_sample_dataset(100)

# Preprocess function
def preprocess_function(examples):
    encoded = tokenizer(examples['text1'], examples['text2'],
                        padding='max_length', truncation=True, max_length=512)
    return encoded

# Apply preprocessing
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    no_cuda=not torch.cuda.is_available()
)

# Metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'precision': precision_score(p.label_ids, preds),
        'recall': recall_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds)
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Plagiarism checker function
def check_plagiarism(text1, text2, threshold=0.8):
    inputs = tokenizer(text1, text2, return_tensors='pt',
                      max_length=512, truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    similarity = probs[0][1].item()
    return {
        "similarity_score": similarity,
        "is_plagiarized": similarity > threshold,
        "threshold": threshold
    }

# Gradio Interface function
def plagiarism_interface(text1, text2, threshold):
    result = check_plagiarism(text1, text2, threshold)
    return result['similarity_score'], result['is_plagiarized']

# Create Gradio App
iface = gr.Interface(
    fn=plagiarism_interface,
    inputs=[
        gr.Textbox(lines=5, placeholder="Enter original text here..."),
        gr.Textbox(lines=5, placeholder="Enter suspicious text here..."),
        gr.Slider(minimum=0.0, maximum=1.0, step=0.01, value=0.8, label="Threshold")
    ],
    outputs=[
        gr.Number(label="Similarity Score"),
        gr.Label(label="Plagiarized?")
    ],
    title="BERT-Plag: Plagiarism Detector",
    description="Enter two texts to check for plagiarism. Adjust the threshold if needed."
)

iface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: heydarshan2180 (heydarshan2180-chandigarh-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://24dd7c8127e6c09123.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Test Case 1 (Plagiarized - Verbatim Copy 80%)

Original:

"Technology has fundamentally reshaped the way humans interact, communicate, and entertain themselves, building bridges that transcend geography and culture."

Suspicious:

"Technology has fundamentally reshaped the way humans interact, communicate, and entertain themselves, building bridges that transcend geography and culture."

Test Case 2 (Plagiarized - Paraphrase Copy)

Original:

"The economy thrives when innovation is prioritized, allowing new ideas to transform industries and create fresh opportunities."

Suspicious:

"Economic growth happens when creativity is encouraged, letting novel concepts change sectors and open new chances."

Test Case 3 (Plagiarized - Minor Verbatim with tweaks)

Original:

"Climate change poses an unprecedented threat to biodiversity, ecosystems, and human societies worldwide."

Suspicious:

"Climate change threatens biodiversity, ecosystems, and human communities across the globe."

Test Case 4 (Plagiarized - AI-Generated Style Change)

Original:

"Learning is a lifelong process that stretches far beyond classrooms, textbooks, and lectures."

Suspicious:

"LEARNING IS A LIFELONG PROCESS THAT STRETCHES FAR BEYOND CLASSROOMS, TEXTBOOKS, AND LECTURES."

✅ Non-plagiarized examples:
Test Case 5 (Not Plagiarized)

Original:

"The stars above painted a canvas of light, shimmering like jewels scattered across a vast velvet sky."

Suspicious:

"A quiet river reflected the moon’s soft glow, carrying whispers of the night into distant dreams."

Test Case 6 (Not Plagiarized)

Original:

"A skilled gardener knows patience, nurturing every seed with care until it blossoms into vibrant life."

Suspicious:

"The persistence of a mountain climber mirrors the quiet strength required to conquer life's steepest peaks."

Test Case 7 (Not Plagiarized)

Original:

"History is written not just by victors but by countless voices who refuse to be forgotten."

Suspicious:

"Art preserves the essence of human emotions, giving voice to the silent stories etched in time."

Test Case 8 (Not Plagiarized)

Original:

"Cities are living organisms, growing, evolving, and adapting to the needs and dreams of their inhabitants."

Suspicious:

"Forests breathe life into the planet, offering shelter, nourishment, and balance to the world."